In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -uq 'drive/MyDrive/Heart Disease/echocardiography.zip' -d 'drive/MyDrive/Heart Disease/dataset/'

In [3]:
validation_path = 'drive/MyDrive/Heart Disease/dataset/validation' 

In [17]:
import os
import numpy as np
from PIL import Image

import tensorflow as tf

import functools

## Set Dataset

In [6]:
# Validation dataset A2C
X_val_A2C = []
Y_val_A2C = []
for i in range(801, 901):
    data_path = validation_path + '/' + 'A2C' + '/'+ '0'* (4-len(str(i))) + str(i) + '.png'
    label_path = validation_path + '/' + 'A2C' + '/'+ '0'* (4-len(str(i))) + str(i) + '.npy' 
    image = Image.open(data_path)
    image = np.array(image.resize((640, 480)))
    X_val_A2C.append(image)

    label = np.load(label_path)
    label = Image.fromarray(label)
    label = np.array(label.resize((640, 480)))
    Y_val_A2C.append(label)
X_val_A2C = np.array(X_val_A2C)
X_val_A2C = X_val_A2C[:,:,:,:3]
Y_val_A2C = np.array(Y_val_A2C)
Y_val_A2C = Y_val_A2C.reshape(100, 480, 640, 1)
print(X_val_A2C.shape, Y_val_A2C.shape)

(100, 480, 640, 3) (100, 480, 640, 1)


In [7]:
# Validation dataset A4C
X_val_A4C = []
Y_val_A4C = []
for i in range(801, 901):
    data_path = validation_path + '/' + 'A4C' + '/'+ '0'* (4-len(str(i))) + str(i) + '.png'
    label_path = validation_path + '/' + 'A4C' + '/'+ '0'* (4-len(str(i))) + str(i) + '.npy' 
    image = Image.open(data_path)
    image = np.array(image.resize((640, 480)))
    X_val_A4C.append(image)

    label = np.load(label_path)
    label = Image.fromarray(label)
    label = np.array(label.resize((640, 480)))
    Y_val_A4C.append(label)
X_val_A4C = np.array(X_val_A4C)
X_val_A4C = X_val_A4C[:,:,:,:3]
Y_val_A4C = np.array(Y_val_A4C)
Y_val_A4C = Y_val_A4C.reshape(100, 480, 640, 1)
print(X_val_A4C.shape, Y_val_A4C.shape)

(100, 480, 640, 3) (100, 480, 640, 1)


# DataLoader

In [12]:
batch_size = 16

In [13]:
def _augment(img,
             label_img,
             resize=None,  # Resize the image to some size e.g. [256, 256]
             scale=1,  # Scale image e.g. 1 / 255.
             hue_delta=0.01,  # Adjust the hue of an RGB image by random factor
             horizontal_flip=True,  # Random left right flip,
             width_shift_range=.1,  # Randomly translate the image horizontally
             height_shift_range=.1):  # Randomly translate the image vertically
    if resize is not None:
        # Resize both images
        label_img = tf.image.resize(label_img, resize)
        img = tf.image.resize(img, resize)
  
    if hue_delta:
        img = tf.image.random_hue(img, hue_delta)
  
    img, label_img = flip_img(horizontal_flip, img, label_img)                               
    img, label_img = shift_img(img, label_img, width_shift_range, height_shift_range)
    label_img = tf.cast(label_img, dtype=tf.float32) * scale
    img = tf.cast(img, dtype=tf.float32) * scale
    return img, label_img

In [14]:
def _process_pathnames(X, Y):
    img = tf.convert_to_tensor(X)
    label = tf.convert_to_tensor(Y)
    return img, label

In [49]:
def gen_test_a2c():
    for x, y in zip(X_val_A2C, Y_val_A2C):
        yield _process_pathnames(x, y)

def gen_test_a4c():
    for x, y in zip(X_val_A4C, Y_val_A4C):
        yield _process_pathnames(x, y)

def get_dataset( 
            preproc_fn = functools.partial(_augment),
            threads = 2,
            batch_size = batch_size,
            test = 'A2C'):
  if test == 'A2C':
      dataset = tf.data.Dataset.from_generator(gen_test_a2c, (tf.float32, tf.float32), ((480, 640, 3), (480, 640, 1)))
  else:
      dataset = tf.data.Dataset.from_generator(gen_test_a4c, (tf.float32, tf.float32), ((480, 640, 3), (480, 640, 1)))
  dataset = dataset.batch(batch_size)
  return dataset

In [50]:
test_dataset_a2c = get_dataset(X_val_A2C, Y_val_A2C, test='A2C')
test_dataset_a4c = get_dataset(X_val_A4C, Y_val_A4C, test='A4C')

In [51]:
test_dataset_a2c.take(1)

<TakeDataset shapes: ((None, 480, 640, 3), (None, 480, 640, 1)), types: (tf.float32, tf.float32)>

## Model Load

In [32]:
model_a2c = tf.keras.models.load_model('./my_model_A2C')
model_a4c = tf.keras.models.load_model('./my_model_A4C')

## Evaluate

In [34]:
def dice_coeff(y_true, y_pred): 
    smooth = 1e-10        
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(tf.square(y_true_f)) + \
                                            tf.reduce_sum(tf.square(y_pred_f)) + smooth)
    return score

In [35]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [36]:
def bce_dice_loss(y_true, y_pred):
    loss = tf.reduce_mean(losses.binary_crossentropy(y_true, y_pred)) + \
          dice_loss(y_true, y_pred)
    return loss

In [37]:
def JI(labels, predictions):
    dsc_loss = bce_dice_loss(labels, predictions)
    loss = dsc_loss/(2-dsc_loss)
    return loss

### A2C Evaluate

In [38]:
mean =[]
for images, labels in test_dataset:
    predictions = model_a2c(images, training=False)
    m = JI(labels, predictions)
    mean.append(m)

In [ ]:
mean = np.array(mean)
mean = np.mean(mean)
print("mean_iou: {}".format(mean))

### A3C Evaluate

In [ ]:
mean =[]
for images, labels in test_dataset:
    predictions = model_a2c(images, training=False)
    m = JI(labels, predictions)
    mean.append(m)

In [ ]:
mean = np.array(mean)
mean = np.mean(mean)
print("mean_iou: {}".format(mean))